<a href="https://colab.research.google.com/github/Hsiao1027/RAG-implement/blob/main/RAG%E6%95%B4%E5%90%88LineAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image](https://drive.google.com/uc?export=view&id=1ziLgQw2nTTcsM6dLg1QW7jFltE_zY3rt)

![image](https://drive.google.com/uc?id=1gO-oKCBhHEKrsoK6KUcaKay3Ewqx4N2r)




In [ ]:
!pip install -q PyPDF2
!pip install -q docling
!pip install -q langchain-text-splitters
!pip install -q rich
!pip install -q faiss-cpu
!pip install -q langchain_huggingface
!pip install -q sentence_transformers
!pip install -q langchain-community
!pip install -q groq
!pip install -q langchain-groq
!pip install -q pyngrok
!pip install -q Flask
!pip install -q line-bot-sdk
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.5/146.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/47

In [ ]:
import os
from google.colab import userdata
from rich.pretty import pprint

In [ ]:
import PyPDF2

def extract_pages(input_pdf_path, output_pdf_path, pages_to_keep):
    """
    從 input_pdf_path 讀取 PDF，並僅保留 pages_to_keep 指定的頁面，輸出至 output_pdf_path。
    參數 pages_to_keep 為一個包含頁碼的清單，頁碼預設為 1-indexed。
    """
    # 讀取 PDF 檔案
    pdf_reader = PyPDF2.PdfReader(input_pdf_path)
    pdf_writer = PyPDF2.PdfWriter()

    # 逐頁加入保留的頁面，注意清單中的頁碼為 indexed-1，因此需減 1
    for page_num in pages_to_keep:
        index = page_num - 1
        if index < len(pdf_reader.pages):
            pdf_writer.add_page(pdf_reader.pages[index])
        else:
            print(f"警告: 頁碼 {page_num} 超出文件總頁數 {len(pdf_reader.pages)}，已忽略。")

    # 將新的 PDF 寫入檔案
    with open(output_pdf_path, "wb") as output_pdf_file:
        pdf_writer.write(output_pdf_file)
    print("PDF 頁面提取完成，輸出檔案：", output_pdf_path)

if __name__ == '__main__':
    # 輸入與輸出 PDF 檔案路徑
    input_pdf = "/content/sample_data/KL-CX-04-2-09-H.pdf"    # 請將此路徑更改為你的 PDF 檔案位置
    output_pdf = "/content/sample_data/output.pdf"  # 輸出檔案名稱
    # 指定要保留的頁數 (indexed-1)，例如：保留第 1、3、5 頁
    pages = [28, 48, 54]

    extract_pages(input_pdf, output_pdf, pages)


PDF 頁面提取完成，輸出檔案： /content/sample_data/output.pdf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import FigureElement, InputFormat, Table
from pathlib import Path


source = "/content/sample_data/output.pdf"  # document per local path or URL
output_dir = Path("/content/sample_data/output")
output_dir.mkdir(parents=True, exist_ok=True)

IMAGE_RESOLUTION_SCALE = 2.0

pipeline_options = PdfPipelineOptions()
pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
pipeline_options.generate_page_images = True # 記憶圖片位置
pipeline_options.generate_picture_images = True

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)
result = doc_converter.convert(source)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
content_md = result.document.export_to_markdown(image_mode=ImageRefMode.REFERENCED)
print(content_md)

<!-- image -->

## 第４章溫控群組

## 1. 料管溫度操作及設定 : 根目錄下＜溫度→溫度設定＞

- (1) 各段溫控溫度與狀態

H1= 射嘴溫控。

- H2= 射嘴延長溫控。
- H3= 料鍋溫控。
- (2) 設定各段溫度目標與偏差限制。
- (3) 設定各段溫度保溫溫度。
- (4) 設定各段溫度是否使用及檢查 注意。 : OFF 時不加溫也不檢查溫度，

## 膠泵可直接動作。

- (5) 設定每日自動開電加熱的時間。
- (6) 設定保溫開關。

<!-- image -->

<!-- image -->

(b)

時間設定 設定各時間參數 :

- 2. 操作登入：於開機首頁選取＜操作登入＞
- (1) 輸入本公司給予您的使用者 ID 與密碼變更使用權限

管理者－ ID:1 PW:2222

產線使用者－ ID:0 無需密碼 僅能操作不能修改任何參數

- (2) 依權限開放設定功能，達到管理的效果。

<!-- image -->

<!-- image -->

## 7. 生產管理

(1) 設定射出各段壓力 kg/cm2 。

設定射出各段速度 。 %

設定射出各段所需時間。

(2) 設定座進時射出馬達預先轉動建立壓力 幫助第一段射出將料順利 (

送出 )

(3) 調整座退方式

(4)

密度：低壓料的料性都在 0.98

常數：為變動的數字，可依照當下射出實際重量回推常數

成品重量：射出料的總重量 不含元件 ( )

射出切換段：實際需要的射出段數

射出重量：程式會自行計算

保壓重量：成品重量 射出重量 -

射出：射出佔總重量的 數字 %

保壓：

100- 射出

(5) 冷卻 ：在機台待機時，自循環所需的扭力跟速度 T

TM 為冷卻設定的時間

(6) 可以查看生產數量，及產品數量到達時提醒警報

## (7)T= 扭力設定

Tf= 實際扭力的回授

TM= 曲線時間範圍

V= 速度

Vf= 速度回授

<!-- image -->


In [ ]:
picture_counter = 0
doc_filename = result.input.file.stem
for element, _level in result.document.iterate_items():

    if isinstance(element, PictureItem):
        picture_counter += 1
        element_image_filename = (
            output_dir / f"{doc_filename}-picture-{picture_counter}.png"
        )
        with element_image_filename.open("wb") as fp:
            element.get_image(result.document).save(fp, "PNG")

![image](https://drive.google.com/uc?id=1hEqbeB3WNMtNhCp_PNNruzAeA4r8voae)


In [ ]:
import re

def replace_images(markdown_text, output_dir, doc_filename):
    # 使用可變型 counter 列表來記錄 index
    counter = [1]

    # 定義替換函數
    def repl(match):
        replacement = f"<!-- image -->[{output_dir}/{doc_filename}-picture-{counter[0]}.png]"
        counter[0] += 1
        return replacement

    # 用 re.sub 進行替換，模式匹配 <!-- image -->（允許左右空格）
    new_text = re.sub(r'<!--\s*image\s*-->', repl, markdown_text)
    return new_text
content_with_image_md = replace_images(content_md, output_dir, doc_filename)
print(content_with_image_md)

<!-- image -->[/content/sample_data/output/output-picture-1.png]

## 第４章溫控群組

## 1. 料管溫度操作及設定 : 根目錄下＜溫度→溫度設定＞

- (1) 各段溫控溫度與狀態

H1= 射嘴溫控。

- H2= 射嘴延長溫控。
- H3= 料鍋溫控。
- (2) 設定各段溫度目標與偏差限制。
- (3) 設定各段溫度保溫溫度。
- (4) 設定各段溫度是否使用及檢查 注意。 : OFF 時不加溫也不檢查溫度，

## 膠泵可直接動作。

- (5) 設定每日自動開電加熱的時間。
- (6) 設定保溫開關。

<!-- image -->[/content/sample_data/output/output-picture-2.png]

<!-- image -->[/content/sample_data/output/output-picture-3.png]

(b)

時間設定 設定各時間參數 :

- 2. 操作登入：於開機首頁選取＜操作登入＞
- (1) 輸入本公司給予您的使用者 ID 與密碼變更使用權限

管理者－ ID:1 PW:2222

產線使用者－ ID:0 無需密碼 僅能操作不能修改任何參數

- (2) 依權限開放設定功能，達到管理的效果。

<!-- image -->[/content/sample_data/output/output-picture-4.png]

<!-- image -->[/content/sample_data/output/output-picture-5.png]

## 7. 生產管理

(1) 設定射出各段壓力 kg/cm2 。

設定射出各段速度 。 %

設定射出各段所需時間。

(2) 設定座進時射出馬達預先轉動建立壓力 幫助第一段射出將料順利 (

送出 )

(3) 調整座退方式

(4)

密度：低壓料的料性都在 0.98

常數：為變動的數字，可依照當下射出實際重量回推常數

成品重量：射出料的總重量 不含元件 ( )

射出切換段：實際需要的射出段數

射出重量：程式會自行計算

保壓重量：成品重量 射出重量 -

射出：射出佔總重量的 數字 %

保壓：

100- 射出

(5) 冷卻 ：在機台待機時，自

![image](https://drive.google.com/uc?id=147pNcwTvBss9kLhAzKraqjng6TssX6BG)

In [ ]:
md_filename =  f"/content/sample_data/{doc_filename}.md"
with open(md_filename,"w", encoding='utf-8') as fp:
    fp.write(content_with_image_md)

![image](https://drive.google.com/uc?id=1rG_nKA5OgYlazga2TcI6h_IFPcCBJd7-)

![image](https://drive.google.com/uc?id=1tPqTZEBmfd01Ri9oPVDWWXg8Tunin2o5)


![image](https://drive.google.com/uc?id=1n_ZatO0IySeu-Nb-FejyntKxjUFnSB3G)


In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from rich.pretty import pprint

# 設定標題分割規則
headers_to_split_on = [
    ("#", "title"),
    ("##", "subtitle"),
]

# 先用 MarkdownHeaderTextSplitter 依照標題切分
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
chunks = markdown_splitter.split_text(content_with_image_md)

# 再用 RecursiveCharacterTextSplitter 進一步切細
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,  # 每個 chunk 最多 300 字符
    chunk_overlap=50  # 避免語意斷裂
)

smaller_chunks = []
for chunk in chunks:
    split_chunks = text_splitter.split_text(chunk.page_content)
    smaller_chunks.extend(split_chunks)

# 輸出每個 chunk
for idx, chunk in enumerate(smaller_chunks):
    print(f"Chunk {idx + 1}: ({len(chunk)} characters)")
    pprint(chunk)


Chunk 1: (64 characters)


'<!-- image -->[/content/sample_data/output/output-picture-1.png]'

Chunk 2: (94 characters)


'- (1) 各段溫控溫度與狀態  \nH1= 射嘴溫控。  \n- H2= 射嘴延長溫控。\n- H3= 料鍋溫控。\n- (2) 設定各段溫度目標與偏差限制。\n- (3) 設定各段溫度保溫溫度。'

Chunk 3: (80 characters)


...

'(5) 冷卻 ：在機台待機時，自循環所需的扭力跟速度 T  \nTM 為冷卻設定的時間  \n(6) 可以查看生產數量，及產品數量到達時提醒警報'

Chunk 18: (43 characters)


'Tf= 實際扭力的回授  \nTM= 曲線時間範圍  \nV= 速度  \nVf= 速度回授'

Chunk 19: (96 characters)


'TM= 曲線時間範圍  \nV= 速度  \nVf= 速度回授  \n<!-- image -->[/content/sample_data/output/output-picture-6.png]'

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings_model = HuggingFaceEmbeddings(model_name="chuxin-llm/Chuxin-Embedding")
vectorstore = FAISS.from_documents(documents=chunks, embedding=embeddings_model)

![image](https://drive.google.com/uc?id=1K5584UED0VTeR60DNr4IoEqjQyzb7UpU)


In [ ]:
# 圖片摘要後存入向量資料庫
from langchain.schema.messages import HumanMessage, SystemMessage
import base64
import uuid
from langchain.schema.document import Document
from langchain_groq import ChatGroq


os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

llm = ChatGroq(
     model="llama-3.2-90b-vision-preview",
     temperature=0,
     max_tokens=None,
     timeout=None,
     max_retries=2,
 )

documents = []
image_elements_SMP = []
image_summaries_SMP = []
output_path = '/content/sample_data/output'
def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')

def summarize_image(encoded_image):
    prompt = [
        HumanMessage(content=[
            {
                "type": "text",
                "text": "你是一個善於分析圖片的助理，以下為製程的流程圖.請用繁體中文詳細解釋這張流程圖的內容."
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                },
            },
        ])
    ]
    response = llm.invoke(prompt)
    return response.content

for i in os.listdir(output_path):
    if i.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(output_path, i)
        encoded_image = encode_image(image_path)
        image_elements_SMP.append(encoded_image)
        summary = summarize_image(encoded_image)
        image_summaries_SMP.append(summary)

for e, s in zip(image_elements_SMP, image_summaries_SMP):
    i = str(uuid.uuid4())
    doc = Document(
        page_content = s,
        metadata = {
            'id': i,
            'type': 'image',
            'original_content': e
        }
    )
    documents.append(doc)

In [ ]:
for idx,i in enumerate(documents):
    pprint(i)

Document(
│   metadata={
│   │   'id': '9686221c-82d7-47b4-80e0-2b6c6fde485c',
│   │   'type': 'image',
│   │   'original_content': 'iVBORw0KGgoAAAANSUhEUgAAAFUAAAAbCAIAAACCxVQRAAAHBElEQVR4nM1Za3MUxxU953bProQkQBIYsJM4NglUbCpx7CSuVKryIX85VXGl8iEPPxJsCBa2MRArD2PJCPRgdyXtzqPvyYeeXSELFFKVCnt2avfundneOd331XcoCRmCIFEQCYouyGAUgAQSModIQAQIAQTaNwfzQCQoCSSVR3UnARqmEfGIRmJmRoIQ0RIlQIGQDAQgomVoAEjlyaDgoAMh/9Ytkyfg/1dmz4YD/kJeytYcKINAAaTMBBhkeTaQkK0EBOAHcwMxjYeTAEIG5XHy/EwbWv4CRAiw7AcQQcDGJ50owZJwwAGBJDpkISRDBYwNXAADINDB9NioJPP6czzm4zKPaI6X9cxXTmQ8QSP7pv1TgRRangBNADXw6gtVa2ADT6CEWdp5zi6j3lC9BroY0HSAGXbmEYLKTWA/e0AbL3j0Dp43f3YO8c8+3HqyJEhwUmq+3r3/O+/fMh+ZIYklz8wt/fJE7A3X30+DT53JIaYFt/Oz516BBuWDz8mhG0we5Mr+MVUuIGdcOMz/kEhAohv26v61ZvPXofwyekNYwmyY/UH3ROH9lebBO1Z9GSmny0/a/Jum2ebhu0V/1a0BaQmGWoSmiz0gZ7HY8mfr84AsZ7gcDo2NV6v11rtF9XnBQWGCB7cLM8svhm5V3v8oVHe73DPKpSrOxdNdlP8Mu7eitpxwgsYAJ5Td6fnRPQqBKR76fnB/7WrRB/WjP/rg/Q76gAsQImdeLc7+NA1u1/0bhfWzcwtR8xdt7nSz9idLj8xSDqGPH4ddcRqgQ1XJ2AZcdNGBkUZfNFt/CdV6FAxIimW4UJx9g1ZUDz+y5qFBMjWwpjjXWXzZ97e0v07WyvledIM0fcTHmKw/AQMFCFKb5LRVP3rP+h+HtA8EMzSc9bkrYem1Zvt20795AiNYbFg3ocOFy1bMjO7fiNgjKYk0ysTUVoLtv0wPBDBC41QxKVhlMqf2NLyTdv4cmq/BxkGmoHi+u/gmnOX29ahHUg0nGNE5N3PylTS47/U9sGRbVAsQ2wpyeu1fgJTTHdoDLqbNZuc97H5s2M2JsFGX8691ly7VO6va+yRgJApI8Pli7lJgp+qtgjukjwtJEbJpo3wYETAg5ZqCkxLAKx9+UW9/ENKmKLgJReq80Dn3lvtuuXU1+BboEqCud17snnyl2lvzat2YKLItiAX62KgwfdXfQYULwElRpEA1aB7Umx9q/zPTEDDCGi1g8Yfx1HfLrZvaXwlsRBmZcDqcvuyhO+zfNW11BPOJFR3GeNOoI/JRzfHys185kZ+o0WPxL0c+BxK476Obzc4HMfUI0M1pqXNhdvnnPtyqNv9acGB5c6Og7kvx5MtlbxX1VwU9ihBJV14kQVQb/jTJqm3Mmcg8ojle1lPGOUY+tP7j+6FgB/agPCO10lq18wcvPyVG2YsbzRfLP7ITL1f3r8Xy7wEukGLSYrF0GSmpd6vDXUPOnxKktivQEue0HlHAwT6PAvZ9b6XsXTVsuCWKZFD3YufMW2m05o9WonacyYxNipi9GBfOlZu3UW8HEqLkIGRwjhMp22A4MbMJ9HThP8rPfiWerhEQQYwDgYAhqnvl1rssVyM9x8QmnC6Wf2xxaXT/ndB8FawR4QoeznQWL6va8/6dQiVlpLctkrGXEmObn6rEPwERAZeDedWwXw+uNzsfdppeBAgkD83cpc6ZK01/zXufFRyaYECNwma+X8wvV5srlh4aG0Jkm0IpM0B0Aqax003lFBjg47BQq75XPfi9lf+IqOmEB7eznTM/oXXr7RtWrxONCIrimWLpSqp6o8EquQ/URJPtS2OulvseOac8V5LHwFxOkRJtMz26mnrXI/qkANXo6sSleOpStXsPeysd7Jq5O5LPFvOXbf7CsPcv+IbopNzkhBtkgDkoZUcYe8R0IhIGARyqvDXa+G3QPWMjhxg9Xui+8DM31A9vxOZeQAkgkSheKhav1MMHVf/uDPb5WJzPO6jMGuNyOn/wSQHpeaLdzCJQAgejrWsYfsJQIQUp1DyFudc5d6nZvsPdv5knMsiZMBfmX2dxLm38puM7tEDSve1+atw6mNCdNMyz9L8p6P7bgvFxzgcaQTFCBLze39hcvzvXRPFs9CB1vXNh4ewv6nSy92DDKkt23tg0iqnz7cXlt4cD7vYGEQv0kwIMhAMKoOAJIEQiCRRCfnxAEGrr67a9MpY5LpKeePaonF/PcuVEzjikcVk4FbM6eRkXvgP8yjAEglBY91u28FaqSp56lfMnHHWiHIwz3+PcG/XwS1t+G9wDozyCdqi9rwDk/i+AAAD0Y/L288n/koc5ehKVwHVoA6yBEghAAE8BS8AQ2oaVcM8PRKAlYB5pgLAD1IABxTcHb/nnnYABxCQeTBEEFP8GpHqfz5hgTlkAAAAASUVORK5CYII='
│   },
│   page_content='這張流程圖顯示了一個簡單的製程流程。以下是每個步驟的詳細解釋：\n\n1.  **開始**：流程從「開始」開始，表示製程的起點。\n2.  **步驟1**：第一個步驟是「步驟1」，這個步驟可能是製程的初始階段，例如原材料的準備或初步處理。\n3.  **步驟2**：第二個步驟是「步驟2」，這個步驟可能是基於步驟1的結果進行進一步的處理或加工。\n4.  **步驟3**：第三個步驟是「步驟3」，這個步驟可能是基於步驟2的結果進行進一步的處理或加工。\n5.  **步驟4**：第四個步驟是「步驟4」，這個步驟可能是基於步驟3的結果進行進一步的處理或加工。\n6.  **步驟5**：第五個步驟是「步驟5」，這個步驟可能是基於步驟4的結果進行進一步的處理或加工。\n7.  **步驟6**：第六個步驟是「步驟6」，這個步驟可能是基於步驟5的結果進行進一步的處理或加工。\n8.  **步驟7**：第七個步驟是「步驟7」，這個步驟可能是基於步驟6的結果進行進一步的處理或加工。\n9.  **步驟8**：第八個步驟是「步驟8」，這個步驟可能是基於步驟7的結果進行進一步的處理或加工。\n10. **步驟9**：第九個步驟是「步驟9」，這個步驟可能是基於步驟8的結果進行進一步的處理或加工。\n11. **步驟10**：第十個步驟是「步驟10」，這個步驟可能是基於步驟9的結果進行進一步的處理或加工。\n12. **步驟11**：第十一個步驟是「步驟11」，這個步驟可能是基於步驟10的結果進行進一步的處理或加工。\n13. **步驟12**：第十二個步驟是「步驟12」，這個步驟可能是基於步驟11的結果進行進一步的處理或加工。\n14. **步驟13**：第十三個步驟是「步驟13」，這個步驟可能是基於步驟12的結果進行進一步的處理或加工。\n15. **步驟14**：第十四個步驟是「步驟14」，這個步驟可能是基於步驟13的結果進行進一步的處理或加工。\n16. **步驟15**：第十五個步驟是「步驟15」，這個步驟可能是基於步驟14的結果進行進一步的處理或加工。\n17. **步驟16**：第十六個步驟是「步驟16」，這個步驟可能是基於步驟15的結果進行進一步的處理或加工。\n18. **步驟17**：第十七個步驟是「步驟17」，這個步驟可能是基於步驟16的結果進行進一步的處理或加工。\n19. **步驟18**：第十八個步驟是「步驟18」，這個步驟可能是基於步驟17的結果進行進一步的處理或加工。\n20. **步驟19**：第十九個步驟是「步驟19」，這個步驟可能是基於步驟18的結果進行進一步的處理或加工。\n21. **步驟20**：第二十個步驟是「步驟20」，這個步驟可能是基於步驟19的結果進行進一步的處理或加工。\n22. **步驟21**：第二十一個步驟是「步驟21」，這個步驟可能是基於步驟20的結果進行進一步的處理或加工。\n23. **步驟22**：第二十二個步驟是「步驟22」，這個步驟可能是基於步驟21的結果進行進一步的處理或加工。\n24. **步驟23**：第二十三個步驟是「步驟23」，這個步驟可能是基於步驟22的結果進行進一步的處理或加工。\n25. **步驟24**：第二十四個步驟是「步驟

Document(
│   metadata={
│   │   'id': 'ae305e84-a66d-4935-9e8b-9c25a915349d',
│   │   'type': 'image',
│   │   'original_content': 'iVBORw0KGgoAAAANSUhEUgAAApUAAABICAIAAAAoDJkoAACLV0lEQVR4nO29d3hU17U+vPaZ3jUqo957BYFACEQTQnQwzYAxNu6OndiJ7Ti5Tm6c6+TefLYTO3FscMWNaowNpoqOQCAJ9d57n5FmRtPLOfv7Y9uTiRpqYJwf7/Mkjxmd2WefMnvttda73oUwxnAP93AP93AP93APPymwf+wJ3MNPHNgO2A7AYMwAYAAGMA0wyqaQAkQBUOS/EWIBYgGiAFjjPK8NMD3M54gLiBrPOPbhJoyA4gKg8U3pHu7hHu7hDuJf9htjPOqye2ugca2bTsDk/6YyEkCGohBCk78uAPTDODDxoRAi/5sMJjsHQACAJjeLf53brsGWVrA0g6UF6AFs6wLaCLQebD2AzSNMEgFLCmw3oISAOMBxQ2x34HgD1wPxI4HrRyYIt7ScmGE0Z8FUBYg16HPkthHxgsZ6MYyNMRSCoRiw4V/nxAxwPJDrfYglHes493AP93APdxxO/jetZtr+B+iBH3yjsQMDS4iki8FlBWIJxzsDDADWHqw+gfV5gG1T4/QgLhJPR/J1mOMFtm6sOogNpeNzy74Hg8RzkNsG4HgAQmBpxaqD2FQ9gUki2UIkXw1s1/HPwXkUBOZWpu8AmGrGPweE5CuQy3JgiSZ8foxpMDdgYyk2N4KxFMzV2G4ARg/YCowJMA3YCox9tA0GAqC4gNgAFFBcTPGBEiLEw7wAEEQjXhASxoFkDlCCUSfC4P7jWH0U0L8HkDDNEs2EsdtvsIPuBqP8FGzKf70emAZBFEu6GO7Z73u4h3u4i+G0/DF6rPwUrLqJGFCWACghki0BGLf9BsaCVQeZ7n+CuWGyfjIBAiSeBbIlwJIAYEyrseYU7j8/kevCAAo9ckkHjgcAgL0Pq49jzbUJDIUNeRRQ4HY/orjjn8f3QADYrsL9x0B7YyKXw3FDsjSAcdtvjAFsHdhYA8YiPJCNLbVg7QG7+l8RbOfJoFttLbAVsBUAgHHEXgCMFaA9jzkKxAsC6WIkTkLCeOAFjxQtwPY+sPQMPhEGwOZxXRimtWDpBJvqX0NhALbL8MH5e7iHe7iHuwbO7gsCxANqQvab4g32hMYADAC0CfcdxspPwdJw63V/TEAgW4y8nkcuy4mlxIAAcYCa0OAYAHGcvjmJoYw1WPk5EkSCeNb4v+yMCc+BmuAzsqsZTSZozmJjGZiqgdY734/JwnkEbANrB7Z2gP4m5gUh4QwkXwaua9HwQQtqhBdm3GEJQEOHmmAmyBk0Tbe1tRUVFZnNZoSQQqGIj4/38PAY9uD6+vrKykqDweDI1Pj6+s6ePZvP52s0mqKiIqVSSdM0i8Xy8/OLj4+XSCTkiy0tLWVlZTqdDgDYbHZoaGhiYuKwmx6NRlNaWtrZ2Ykx5nA4ERERkZGRPB5vpPl3d3er1erW1laNRkOmhDFmsVhubm4eHh7+/v5yuXzYLzIM09TUVFFRYTAYAEAulyckJHh7ew+aFbk/7e3tKpXKbP5+14UQkkgkXl5eAQEB7u7ut7rHo0GlUimVyra2tr6+PnJqjDFFUb6+vj4+Pt7e3gLB6DGeiaC7u7usrEylUgGAUCiMjo4ODQ1lscbJ7RgVJpOpubnZxcXF29vb8aHZbK6oqGhsbLTb7QihwMDA+Ph4sVg8hee9A+jq6urp6enq6tJoNOSHwGKx5HK5QqEICwsTiSYeOPxPxY/KX8MM1l5iej8AU/UUjUghYRzy/BlyWYEozhSNOVWwYX0Bo8mkBJFASSebCb/9wADAmMDaxvQfx72fgbkGsA3gjpC6sBXMtdhcj42FyKak3LcAL+Cuv2GDYbfbs7OzX3755d7eXhaLlZqa+tprrw1rv5uamt5+++2vv/5arVYDAEIoJibm17/+9axZswCgvb39zTffvHLlis1m43A4Gzdu/MMf/kDsN8MwOTk5f/zjH+vr6xFCHA5nxYoVf/vb3wIDA4eepbW19a233srMzKRpWiQS/eIXv/jlL385rP1Wq9VVVVXfffddc3Nzfn5+Z2cnTdMAQAx/YGBgRETEtGnTFi9enJSUNNRIMAxz5cqVv/zlL83NzQAwffr0P/7xj15eXs72u7u7OzMzMzs7u7S0tKGhYWBggHxOUZRCoYiKipoxY8bixYvnzp07ASNkNpsbGhqOHDlSUVFRWFjY0tLisN8sFmvatGmxsbGLFy9OS0tTKBRs9lSugWVlZa+99lpubi7G2MfH5ze/+U1AQMBU2W+9Xt/b25udnX39+vU1a9Y422+tVvvll19++umnJpOJoqj777//j3/840/Ifvf39zc2Nn799ddlZWXl5eVdXV3EfnM4nICAgNDQ0I0bN6alpXl7e3O5E49f/ufhR7PfGGOwtOCe98FQBNgyFUNSwA9FXr9ELivvPuMNAAB2NVbuwxxf5PEgoLtyhj8AYwC7Cvd/y/R+BsZiYIw/xiwYMNfijj8xpirK5zcgCLnLb9pQ0DRtsVjsdrvdbrfZbENrNTHGbW1tr7766rfffqvX68mHERERTz/99JYtW8i6jzG22WwWi4WmaYZh7Ha78ziOUwCAzWa7fPnynj17fvnLXw51jjHGdrudeLqOrwwCwzDV1dV79uw5cuRIW1sbMdvOsNlsVVVVVVVVx44dO3To0EsvvbRt27ahjtGgWTmPgzFubW199913d+/eTRz0QWhtbW1tbT179uyhQ4d+85vfbN++fVx2yGQyHT169L333svPz7dYBi8sNpstNzc3Nzf3+PHjS5Yseeqpp+bNm8fhTNl7RdO0zWaz2WwAQB7ZVI1sMpkOHDiwa9euqqqq8PDw5cuXO/+VYRiLxWIwGMgZzWYzwzBTderbCoZhGhoadu3adfjw4a6urkHTttls1dXV1dXVFy5cSE5O/tWvfpWRkXE7Aic/Ufw49hsDgKWF6XgDD1wGZjwJyxGBED8Ueb+A3DZOgEN3h4AALI1YtR+Jp4No+l1bnoQBwNrK9HyCez8Gu+r7XPWPBVqP+79hbJ1I8SRyWX73PtzhgBCiKIr8x7Ax7ba2trfffvvEiRMOSxYbG/vLX/5y586dzk4bGYemaYqiBo3jOAXBwMDA8ePHFyxYsHjxYufPnQ9mGGboOATl5eWvv/46mc8tbU9DQ8Pf//53mqYfeuihQUsqxtixEA+69t7e3o8++mjPnj3DGm9ntLa2/uMf/+ByuVu2bBEKx/rcKyoqdu/ePazxdoZKpcrMzDSbzTKZbPr06ZMsynDA+WJHuskTAMMw+fn5n376aXl5OYmQDzqAy+WGhISkpKSYzWaKoiIiIvh8/pSc+najsrLy7bffPnz4sMFgGGXPYTabb9y4odPpDAbDhg0bfipXd7vxI/nfNiVWfon7vwZGNxXDUSCIQl6/QO4PIPbdzRnGVmwoYHo/YwX8GViSH3s2wwMbynD3P3H/UbAp74I9BgZai7WXsF1DYQu4bpwMAfCuQnt7+/vvv793714SNmez2ZGRkb/73e82bdo04aCuzWarq6s7c+ZMbGysl5fX2L+IMa6trX3rrbe+++47RySAoigOh+Pi4kLMp81mGxgYsFgsVquVePNVVVUffvhhQEDA0qVLHXNGCMlksqCgIL1eTwLjzvamsrLy2LFj/f395J9kfOLBI4RomtZoNHq9nmEYmqZra2s//fTTmJiY2bNnj8UWdnV1ffXVV0VFRcR4s1gsDocjkUhEIhEZXKvVmkwmu93OMIxarb5x48bFixdDQ0Ol0rt63cAY63S6/v7+YaMmACCXy5955plHH32UxGZ4PN7Ydzw/Ipqamnbv3n3kyBFC4AAADofD4XBEIpFYLCbvm9lsJrErq9VaWlr68cc

Document(
│   metadata={
│   │   'id': '3d8b4f72-f632-4c3a-8a39-8c416d99e2ab',
│   │   'type': 'image',
│   │   'original_content': 'iVBORw0KGgoAAAANSUhEUgAAAFUAAAAaCAIAAABJmYe0AAAG3UlEQVR4nM1Ya3MUxxU953bProQkhCQwYCchmERUbCpxTBJXKlX5kL+cqrhS+ZAHthNsCBa2MRCUB1gyAj3YXUm78+h7/GFmFjY8QqpSYc9O7d6509XbZ/q++lISagiCRBdAECAAioQABwlZAggZCEAABQAyAKIgACTggAOBTkDJBNBQ66cOcSwJaHkBAGUQKICUmQCDjPXzBEFs3pEDBCgQEFM7ndS8LNXziP9fZi+Hhr8AEQKstgOIIGDtQydyMGeztwJJdMhMSIYCAGjNWAZAoIPpiVlJ1vvPds4nZT6lebGslx45lvEMjSxiElQg1dowQBNADby4o2IDrOAJlDBLO8XZFZRbKjdAFwOqDjDDzjxCUL4NHIqAHGLjUtPGn50J/hTH35AECU5K1df793/n/RvmIzMkMefxueVfHom94eaHafC5MznEtOB2avbkWWiQP/iSHLrB5EGu2j+mygXkjAuT/CdEAhLdcFD2r1Tbvw753egVYQmzYfYH3SOZ99eqB+9bcTdSTpcftfl3TbPVw0tZf92tAmkJhlKEpos9IGe21PBn4/OAjKzDOUAYKy/Wy51LWfFlxkFmgge30zMrr4dukd//JBS3uzwwyqUizsVjXeT/DPs3onaccILGACdUu9Oro/s0BKY4cf94fc1u0Qfloz/64MMO+oALECJn3sxO/DQNbpb9a5n1a+cWoubP2dyxauNPlh6ZJbVZdHxNuuI0QDZx19iAiy46MNLoTrXzl1BsRsGApJiH09mJd2hZ8fATqx4aJFMFq7KTnaUzfrijw02yVJ3vRTdI00e8xXj/CRgoQJCaJKed8tEH1v80pEMgmKHirM9dCMtvVbs3q/71IxjBYsWyCh0unLdsZnT/WsQBSUmkUSYmUk/8y/RAACPUpopmmYJM5tSBhrfS3p9D9TVYOcgUFE91l96FM9+9GvVIKuEEIzonZ46eTYP7Xt4DczZFpQBRfFxbTZ0ZyOpVqk53aC64mLarvQ+w/6lhv06Elbqcf6u7vFrurevgs4CRKCDB57O51cBO0VsH90hvC0kRsmmjPIkIGJDqmoLjEsALH94pdz8KaVsU3IQsdV7rnLzovp/vXA6+A7oEqOud17tHzxYHG15sGhNFNgWxQG+NCtNX/T2ucAE4KYoUqArVg3L7Yx1+YRoCRlilBSz9MC5+N9+5rsO1wEqUkQnHwrHzHrrD/m3TTkcwH1vRJNim2afkpzUvll9+5Fh+pkZPxL868jmQwEMfXa/2PoqpR4BuTkud07MrP/fhTrH914wDqw83Cuq+EY+eyXvrKL/K6FGESLrqTRJENeFP46zaxJyxzKc0L5b1nHleIE/sf7seCvbYHlS/kVJpo9j7g+efE6PaiyvNZys/siNnivtXYv73ABdIMWkpWz6PlNS70eF+fQ5uogmB1uY1WQVM1RUFPD7nUcChH6zlvcuGLbdEkQzqnuscv5hGG/5oLWrPmcxYpYjZc3HhZL59E+VuICFKDkIGZ5tI2QTDsZmNoecL/1F++ZF4vkZABNEGAgFDFPfynUvM12PdvgCrcCxb+bHF5dH990P1VbBKhCt4ON5ZOq/iwPu3MuWUkV4fc9R6KdHa/FQl/jGICLgcrHcNh+XgarX3cafqRYBA8lDNrXaOX6j6G977IuPQBANKZDbz/Wx+pdhes/TQWBEimxRKmQGiEzC1TjeVr8AAb8NCqfJe8eD3lv8joqQTHtxOdI7/hNYtd69ZuUlUIiiKx7PlC6nojQbr5CFQElVtX2q5Wt33qHPKKyX5ApjLKVKibadHl1PvakSfFKASXR1ZjYurxf49HKx1sG/m7kg+m82ft/nTw96/4Fuik3KTE26QAeagVDtC6xHTiUgYBHCo/MZo67dB94yVHGL0eLr72s/cUD68Fqt7ATmARCJ7I1u6UA4fFP3bMzjkE3G+PkHVrNGW0/UPnxWQXiWawywCJXAw2rmC4WcMBVKQQslFzL3NudVq9xb3/2aeyCBnwlyYf5vZybT1m47v0QJJ96b7qbZ1MKarup3QSv+bgu6/LRif5PxYIyhGiICXh1vbm7fnqiieiB6krndOL5z4RZmO9h5sWWHJThmrSjF1vr208t5wwP3eIGKBflSAgXBAARS87pKTSAKFACSQBKGmvm7aK63Mtkh65tOn5frzMiPHco0JjcvCYqzVyfO48B3gV4YhEITMut+yhYupyLn4JuePOMpEORhnvse5d8rhXVt5DzwAozyCNtHeVwDq/i+AAAD0F+TtV5P/JQ9z9CQqgZvQFlgCORCAAC4Cy8AQ2oXlcEd9rtEyMI80QNgDSsCA7N8nb/jXJwEDiHE8mCIIyL4BdVCKv6v/R1MAAAAASUVORK5CYII='
│   },
│   page_content='這張流程圖顯示了一個簡單的製程流程，包括以下步驟：\n\n1.  **開始**：流程從左側的「開始」標誌開始，表示流程的起點。\n2.  **步驟1**：第一個步驟是「步驟1」，表示流程的第一個任務或動作。\n3.  **步驟2**：第二個步驟是「步驟2」，表示流程的第二個任務或動作。\n4.  **步驟3**：第三個步驟是「步驟3」，表示流程的第三個任務或動作。\n5.  **步驟4**：第四個步驟是「步驟4」，表示流程的第四個任務或動作。\n6.  **步驟5**：第五個步驟是「步驟5」，表示流程的第五個任務或動作。\n7.  **步驟6**：第六個步驟是「步驟6」，表示流程的第六個任務或動作。\n8.  **步驟7**：第七個步驟是「步驟7」，表示流程的第七個任務或動作。\n9.  **步驟8**：第八個步驟是「步驟8」，表示流程的第八個任務或動作。\n10. **步驟9**：第九個步驟是「步驟9」，表示流程的第九個任務或動作。\n11. **步驟10**：第十個步驟是「步驟10」，表示流程的第十個任務或動作。\n12. **步驟11**：第十一個步驟是「步驟11」，表示流程的第十一個任務或動作。\n13. **步驟12**：第十二個步驟是「步驟12」，表示流程的第十二個任務或動作。\n14. **步驟13**：第十三個步驟是「步驟13」，表示流程的第十三個任務或動作。\n15. **步驟14**：第十四個步驟是「步驟14」，表示流程的第十四個任務或動作。\n16. **步驟15**：第十五個步驟是「步驟15」，表示流程的第十五個任務或動作。\n17. **步驟16**：第十六個步驟是「步驟16」，表示流程的第十六個任務或動作。\n18. **步驟17**：第十七個步驟是「步驟17」，表示流程的第十七個任務或動作。\n19. **步驟18**：第十八個步驟是「步驟18」，表示流程的第十八個任務或動作。\n20. **步驟19**：第十九個步驟是「步驟19」，表示流程的第十九個任務或動作。\n21. **步驟20**：第二十個步驟是「步驟20」，表示流程的第二十個任務或動作。\n22. **步驟21**：第二十一個步驟是「步驟21」，表示流程的第二十一個任務或動作。\n23. **步驟22**：第二十二個步驟是「步驟22」，表示流程的第二十二個任務或動作。\n24. **步驟23**：第二十三個步驟是「步驟23」，表示流程的第二十三個任務或動作。\n25. **步驟24**：第二十四個步驟是「步驟24」，表示流程的第二十四個任務或動作。\n26. **步驟25**：第二十五個步驟是「步驟25」，表示流程的第二十五個任務或動作。\n27. **步驟26**：第二十六個步驟是「步驟26」，表示流程的第二十六個任務或動作。\n28. **步驟27**：第二十七個步驟是「步驟27」，表示流程的第二十七個任務或動作。\n29. **步驟28**：第二十八個步驟是「步驟28」，表示流程的第二十八個任務或動作。\n30. **步驟29**：第二十九個步驟是「步驟29」，表示流程的第二十九個任務或動作。\n31. **步驟30**：第三十個步驟是「步驟30」，表示流程的第三十個任務或動作。\n32. **步驟31**：第三十一個步驟是「步驟31」，表示流程的第三十一個任務或動作。\n33. **步驟3

Document(
│   metadata={
│   │   'id': 'a291d8dc-cc55-4a77-8652-10f0ead28462',
│   │   'type': 'image',
│   │   'original_content': 'iVBORw0KGgoAAAANSUhEUgAAApQAAABICAIAAADHzvIWAACLUElEQVR4nO29d3hU17U+vPaZ3jUqo957BYFACEQTQlSDaQaMsXF37MRObMfJdXLjXCf35rOd2IljgytuVGNsMFV0BAJJqPfe+4w0M5peztnfH9ueTNRQozg/3udJHjM6s88+Zfbaa613vQthjOEe7uEe7uEe7uEefjpg3+kJ3MNPGdgO2A7AYMwAYAAGMA0wynaQAkQBUOS/EWIBYgGiAFjjPK8NMD3M54gLiBrPOPbhJoyA4gKg8U3pHu7hHu7hNuIH440xHnXNvTnQuBZNJ2Dyf1MZACBDUQihyV8XAPpxHJj4UAiR/00Gk50DIABAk5vFv85t12BLK1iawdIC9AC2dQFtBFoPth7A5hEmiYAlBbYbUEJAHOC4IbY7cLyB64H4kcD1IxOEm5pNzDCaM2CqAsQa9Dly24B4QWO9GMbGGArBUAzY8K9zYgY4Hsj1fsSSjnWce7iHe7iH244fPW9azbT9D9ADP3pFYwcGlhBJF4PLCsQSjvf0GACsPVh9HOvzANumxt1BXCSejuRrMccLbN1YdQAbSsfnkP0ABonnILf1wPEAhMDSilUHsKl6ApNEsoVIvhrYruOfg/MoCMytTN9+MNWMfw4IyVcgl+XAEk34/BjTYG7AxlJsbgRjKZirsd0AjB6wFRgTYBqwFRj7aLsLBEBxAbEBKKC4mOIDJUSIh3kBIIhGvCAkjAPJHKAEo06Ewf3HsPoIoH+PG2GaJZoJYzfeYAfddUb5GdiU/3o9MA2CKJZ0Mdwz3vdwD/dwF+PH5Y/RY+VnYNVNxHqyBEAJkWwJwLiNNzAWrDrAdP8TzA2T9ZAJECDxLJAtAZYEAGNajTUncf+5iVwXBlDokUs6cDwAAOx9WH0Ma65OYChsyKOAArcHEMUd/zx+AALAdhXuPwra6xO5HI4bkqUBjNt4Ywxg68DGGjAW4YFsbKkFaw/Y1f8KXDtPBt1sX4GtgK0AAIwj6gJgrADtOcxRIF4QSBcjcRISxgMveKQ4Abb3gaVn8IkwADaP68IwrQVLJ9hU/xoKA7Bdho/J38M93MM93DVw+C4IEA+oCRlvijfYBxoDMADQJtx3CCs/A0vDzRf9MQGBbDHyegG5LCdmEgMCxAFqQoNjAMRx+uYkhjLWYOUXSBAJ4lnj/7IzJjwHaoLPyK5mNJmgOYONZWCqBlrvfD8mC+cRsA2sHdjaAfobmBeEhDOQfBm4rkHDhyuoEV6YcQckAA0daoIJIGfQNN3W1lZUVGQ2mxFCCoUiPj7ew8Nj2IPr6+srKysNBoMjQePr6zt79mw+n6/RaIqKipRKJU3TLBbLz88vPj5eIpGQL7a0tJSVlel0OgBgs9mhoaGJiYnD7ng0Gk1paWlnZyfGmMPhREREREZG8ni8kebf3d2tVqtbW1s1Gg2ZEsaYxWK5ubl5eHj4+/vL5fJhv8gwTFNTU0VFhcFgAAC5XJ6QkODt7T1oVuT+tLe3q1Qqs/mHLRdCSCKReHl5BQQEuLu73+wejwaVSqVUKtva2vr6+sipMcYURfn6+vr4+Hh7ewsEo0d3JoLu7u6ysjKVSgUAQqEwOjo6NDSUxRonn2NUmEym5uZmFxcXb29vx4dms7mioqKxsdFutyOEAgMD4+PjxWLxFJ73NqCrq6unp6erq0uj0ZAfAovFksvlCoUiLCxMJJp4yPA/FXeOsIYZrL3I9H4IpuopGpFCwjjk+TPksgJRnCkac6pgw/oCRpNJCSKBkk42+33rgQGAMYG1jek/hns/B3MNYBvAbWFxYSuYa7G5HhsLkU1JuW8GXsBdf8MGw263Z2dnv/LKK729vSwWKzU19fXXXx/WeDc1Nb3zzjvffPONWq0GAIRQTEzMr3/961mzZgFAe3v7W2+9dfnyZZvNxuFwNmzY8Ic//IEYb4ZhcnJy/vjHP9bX1yOEOBzOihUr/va3vwUGBg49S2tr69tvv52ZmUnTtEgk+sUvfvHLX/5yWOOtVqurqqq+//775ubm/Pz8zs5OmqYBgFj9wMDAiIiIadOmLV68OCkpaaiFYBjm8uXLf/nLX5qbmwFg+vTpf/zjH728vJyNd3d3d2ZmZnZ2dmlpaUNDw8DAAPmcoiiFQhEVFTVjxozFixfPnTt3AhbIbDY3NDQcPny4oqKisLCwpaXFYbxZLNa0adNiY2MXL16clpamUCjY7KlcAMvKyl5//fXc3FyMsY+Pz29+85uAgICpMt56vb63tzc7O/vatWv33Xefs/HWarVfffXVZ599ZjKZKIp64IEH/vjHP/6EjHd/f39jY+M333xTVlZWXl7e1dVFjDeHwwkICAgNDd2wYUNaWpq3tzeXO/HI5X8e7ozxxhiDpQX3fACGIsCWqRiSAn4o8volcll591luAACwq7FyL+b4Io+HAN2VM/wRGAPYVbj/O6b3czAWA2O8E7NgwFyLO/7EmKoon9+AIOQuv2lDQdO0xWKx2+12u91msw2tycQYt7W1vfbaa999951erycfRkREPPPMM5s3byaLPsbYZrNZLBaaphmGsdvtzuM4TgEANpvt0qVLu3fv/uUvfznULcYY2+124uM6vjIIDMNUV1fv3r378OHDbW1txGY7w2azVVVVVVVVHT169ODBgy+//PLWrVuHukSDZuU8Dsa4tbX1vffe27VrF3HNB6G1tbW1tfXMmTMHDx78zW9+s23btnEZIZPJdOTIkffffz8/P99iGbyw2Gy23Nzc3NzcY8eOLVmy5Omnn543bx6HM2XvFU3TNpvNZrMBAHlkUzWyyWTav3//zp07q6qqwsPDly9f7vxXhmEsFovBYCBnNJvNDMNM1alvKRiGaWho2Llz56FDh7q6ugZN22azVVdXV1dXnz9/Pjk5+Ve/+lVGRsatCJn8RHEHjDcGAEsL0/EmHrgEzHiSlCMCIX4o8n4RuW2YAGnuNgEBWBqxah8STwfR9Lu2EgkDgLWV6fkU934CdtUP+ek7BVqP+79lbJ1I8RRyWX73PtzhgBCiKIr8x7Ch7La2tnfeeef48eMOMxYbG/vLX/5yx44dzu4aGYemaYqiBo3jOAXBwMDAsWPHFixYsHjxYufPnQ9mGGboOATl5eVvvPEGmc9NDU9DQ8Pf//53mqYffvjhQespxtixCg+69t7e3o8//nj37t3DWm5ntLa2/uMf/+ByuZs3bxYKx/rcKyoqdu3aNazldoZKpcrMzDSbzTKZbPr06ZMswXDA+WJHuskTAMMw+fn5n332WXl5OQmMDzqAy+WGhISkpKSYzWaKoiIiIvh8/pSc+lajsrLynXfeOXTokMFgGGXDYTabr1+/rtPpDAbD+vXrfypXd6txJzxvmxIrv8L93wCjm4rhKBBEIa9fIPcHEfvuZghjKzYUML2fswL+DCzJnZ7N8MCGMtz9T9x/BGzKu2CDgYHWYu1FbNdQ2AKuGybD+Lur0N7e/sEHH+zZs4dEy9lsdmRk5O9+97uNGzdOOJZrs9nq6upOnz4dGxvr5eU19i9ijGtra99+++3vv//eEQOgKIrD4bi4uBDbabPZBgYGLBaL1WolfnxVVdVHH30UEBCwdOlSx5wRQjKZLCgoSK/Xk3i4s7GprKw8evRof38/+ScZn/juCCGapjUajV6vZxiGpuna2trPPvssJiZm9uzZYzGEXV1dX3/9dVFREbHcLBaLw+FIJBKRSEQG12q1JpPJbrczDKNWq69fv37hwoXQ0FCp9K5eNzDGOp2uv79/2HgJAMjl8mefffaxxx4jURkejzf27c4dRFNT065duw4fPkxIGwDA4XA4HI5IJBKLxeR9M5vNJGpltVpLS0s/+eS

Document(
│   metadata={
│   │   'id': '42f53913-7d67-467f-8c39-dc6ba6f626fb',
│   │   'type': 'image',
│   │   'original_content': 'iVBORw0KGgoAAAANSUhEUgAAApQAAABICAIAAADHzvIWAACLTUlEQVR4nO29d3hU17U+vPaZ3jUqo957BYFACEQTQnQwzYAxNu6OndiJ7Ti5Tm6c6+TefLYTO3FscMWNaowNpncQCCShgnrvXTPSzGh6OWd/f2x7MhkV1MA4P97nSR4zOrPPPmX22mutd70LYYzhHu7hHu7hHu7hHn46YP/YE7iHnziwHbAdgMGYAcAADGAaYIQdIQWIAqDIfyPEAsQCRAGwxnheG2B6iM8RFxA1lnHsQ00YAcUFQGOb0j3cwz3cw53Cv4w3xnjENffWQGNaNJ2Ayf9NZgyADEUhhCZ+XQDoh3Fg/EMhRP43EUx0DoAAAE1sFv86t12DLa1gaQZLC9AD2NYFtBFoPdh6AJuHmSQClhTYHkAJAXGA44HYnsDxBa4X4kcDN4BMEG5pNjHDaM6AqQoQy+Vz5LEe8UJGezGMjTEUgeEmYMO/zokZ4Hgh9/sQSzrace7hHu7hHu4snDxvWs20/Q/QAz94RaMHBpYQSReC2zLEEo51BhgArD1YfQzr8wHbJsfdQVwknorkazDHB2zdWLUfG0rH5pB9DwaJZyGPdcDxAoTA0opV+7GpehyTRLL5SL4S2O5jn4PzKAjMrUzfPjDVjH0OCMmXIbelwBKN+/wY02BuwMZSbG4EYymYq7HdAIwesBUYE2AasBUY+0i7CwRAcQGxASiguJjiAyVEiId5QSCIRbwQJEwAySygBCNOhMH9R7H6MKB/Dx1hmiWaDqM33mAH3XVG+SnYlP96PTANghiWdCHcM973cA/3cLfCae1j9Fj5KVh147GeLAFQQiRbBDBm4w2MBav2M93/BHPDRD1kAgRIPANki4AlAcCYVmPNCdx/bjzXhQEUeuSWCRwvAAB7H1YfxZqr4xgKG/IpoMDjfkRxxz6P74EAsF2F+4+A9vp4LofjgWQZAGM23hgD2DqwsQaMxXggB1tqwdoDdvW/AtfOk0G32ldgK2ArAADjiLoAGCtAew5zFIgXAtKFSJyChInACx0uToDtfWDpcT0RBsDmMV0YprVg6QSb6l9DYQC229Ax+Xu4h3u4h7sDzo4LAsQDalzGm+K5+kCjAAYA2oT7DmLlp2BpuPWiPyogkC1EPs8jt6XETGJAgDhAjWtwDIA4Tt+cwFDGGqz8HAmiQTxj7F92xrjnQI3zGdnVjOY0aM5gYxmYqoHWO9+PicJ5BGwDawe2doD+BuaFIOE0JF8C7qvR0OEKapgXZswBCUCDhxpnAsgZNE23tbUVFxebzWaEkEKhSExM9PLyGvLg+vr6yspKg8HgSND4+/vPnDmTz+drNJri4mKlUknTNIvFCggISExMlEgk5IstLS1lZWU6nQ4A2Gx2eHh4cnLykDsejUZTWlra2dmJMeZwOFFRUdHR0Tweb7j5d3d3q9Xq1tZWjUZDpoQxZrFYHh4eXl5egYGBcrl8yC8yDNPU1FRRUWEwGABALpcnJSX5+vq6zIrcn/b2dpVKZTZ/v+VCCEkkEh8fn6CgIE9Pz1vd45GgUqmUSmVbW1tfXx85NcaYoih/f38/Pz9fX1+BYOToznjQ3d1dVlamUqkAQCgUxsbGhoeHs1hj5HOMCJPJ1Nzc7Obm5uvr6/jQbDZXVFQ0Njba7XaEUHBwcGJiolgsnsTz3gF0dXX19PR0dXVpNBryQ2CxWHK5XKFQREREiETjDxn+R+JHJaxhBmsvMr0fgKl6kkakkDABef8MuS1DFGeSxpws2LC+kNGcpgTRQEknmv2+/cAAwJjA2sb0H8W9n4G5BrAN4I6wuLAVzLXYXI+NRcimpDw3AS/orr9hrrDb7Tk5OS+//HJvby+LxUpPT3/ttdeGNN5NTU1vv/32119/rVarAQAhFBcX9+tf/3rGjBkA0N7e/uabb16+fNlms3E4nPXr1//hD38gxpthmNzc3D/+8Y/19fUIIQ6Hs2zZsr/97W/BwcGDz9La2vrWW2+dPn2apmmRSPSLX/zil7/85ZDGW61WV1VVfffdd83NzQUFBZ2dnTRNAwCx+sHBwVFRUVOmTFm4cGFKSspgC8EwzOXLl//yl780NzcDwNSpU//4xz/6+Pg4G+/u7u7Tp0/n5OSUlpY2NDQMDAyQzymKUigUMTEx06ZNW7hw4ezZs8dhgcxmc0NDw6FDhyoqKoqKilpaWhzGm8ViTZkyJT4+fuHChRkZGQqFgs2ezDWwrKzstddey8vLwxj7+fn95je/CQoKmizjrdfre3t7c3Jyrl27tmrVKmfjrdVqv/zyy08//dRkMlEUdf/99//xj3/8CRnv/v7+xsbGr7/+uqysrLy8vKurixhvDocTFBQUHh6+fv36jIwMX19fLnf8kcv/MPxoxhtjDJYW3PM+GIoBWyZjSAr44cjnl8ht+d1nuQEAwK7Gyj2Y44+8HgR0V87wB2AMYFfh/m+Z3s/AeBMY448xCwbMtbjjT4ypivL7DQjC7vKbNhg0TVssFrvdbrfbbTbb4LJMjHFbW9urr7767bff6vV68mFUVNTTTz+9adMmsuhjjG02m8VioWmaYRi73e48juMUAGCz2S5durRr165f/vKXg91ijLHdbic+ruMrLmAYprq6eteuXYcOHWprayM22xk2m62qqqqqqurIkSMHDhx46aWXtmzZMtglcpmV8zgY49bW1nfffXfnzp3ENXdBa2tra2vrmTNnDhw48Jvf/Gbr1q1jMkImk+nw4cPvvfdeQUGBxeK6sNhstry8vLy8vKNHjy5atOipp56aM2cOhzNp7xVN0zabzWazAQB5ZJM1sslk2rdv344dO6qqqiIjI5cuXer8V4ZhLBaLwWAgZzSbzQzDTNapbysYhmloaNixY8fBgwe7urpcpm2z2aqrq6urq8+fP5+amvqrX/0qKyvrdoRMfor4cYw3BgBLC9PxBh64BMxYkpTDAiF+OPJ9AXmsHwdp7g4BAVgasWovEk8F0dS7thIJA4C1len5BPd+DHbV9/npHwu0Hvd/w9g6keJJ5Lb07n24QwEhRFEU+Y8hQ9ltbW1vv/32sWPHHGYsPj7+l7/85fbt253dNTIOTdMURbmM4zgFwcDAwNGjR+fNm7dw4ULnz50PZhhm8DgE5eXlr7/+OpnPLQ1PQ0PD3//+d5qmH3roIZf1FGPsWIVdrr23t/ejjz7atWvXkJbbGa2trf/4xz+4XO6mTZuEwtE+94qKip07dw5puZ2hUqlOnz5tNptlMtnUqVMnWILhgPPFDneTxwGGYQoKCj799NPy8nISGHc5gMvlhoWFpaWlmc1miqKioqL4fP6knPp2o7Ky8u233z548KDBYBhhw2E2m69fv67T6QwGw7p1634qV3db8SN53jYlVn6J+78GRjcZw1EgiEE+v0CeDyD23c0QxlZsKGR6P2MF/RlYkh97NkMDG8pw9z9x/2GwKe+CDQYGWou1F7FdQ2ELuK+fCOPvrkJ7e/v777+/e/duEi1ns9nR0dG/+93vNmzYMO5Yrs1mq6urO3XqVHx8vI+Pz+i/iDGura196623vvvuO0cMgKIoDofj5uZGbKfNZhsYGLBYLFarlfjxVVVVH374YVBQ0OLFix1zRgjJZLKQkBC9Xk/i4c7GprKy8siRI/39/eSfZHziuyOEaJrWaDR6vZ5hGJqma2trP/3007i4uJkzZ47GEHZ1dX311VfFxcXEcrNYLA6HI5FIRCIRGVyr1ZpMJrvdzjCMWq2+fv36hQsXwsPDpdK7et3AGOt0uv7+/iHjJQAgl8ufeeaZRx99lERleDze6Lc7PyKampp27tx56NAhQtoAAA6Hw+FwRCKRWCwm75vZbCZRK6vVWlpa+vH

Document(
│   metadata={
│   │   'id': 'c32aa72f-d8f4-4cdd-b686-5e8aae3c4948',
│   │   'type': 'image',
│   │   'original_content': 'iVBORw0KGgoAAAANSUhEUgAAAFUAAAAaCAIAAABJmYe0AAAG3UlEQVR4nM1Ya3MUxxU953bProQkhCQwYCchmERUbCpxTBJXKlX5kL+cqrhS+ZAHthNsCBa2MRCUB1gyAj3YXUm78+h7/GFmFjY8QqpSYc9O7d6509XbZ/q++lISagiCRBdAECAAioQABwlZAggZCEAABQAyAKIgACTggAOBTkDJBNBQ66cOcSwJaHkBAGUQKICUmQCDjPXzBEFs3pEDBCgQEFM7ndS8LNXziP9fZi+Hhr8AEQKstgOIIGDtQydyMGeztwJJdMhMSIYCAGjNWAZAoIPpiVlJ1vvPds4nZT6lebGslx45lvEMjSxiElQg1dowQBNADby4o2IDrOAJlDBLO8XZFZRbKjdAFwOqDjDDzjxCUL4NHIqAHGLjUtPGn50J/hTH35AECU5K1df793/n/RvmIzMkMefxueVfHom94eaHafC5MznEtOB2avbkWWiQP/iSHLrB5EGu2j+mygXkjAuT/CdEAhLdcFD2r1Tbvw753egVYQmzYfYH3SOZ99eqB+9bcTdSTpcftfl3TbPVw0tZf92tAmkJhlKEpos9IGe21PBn4/OAjKzDOUAYKy/Wy51LWfFlxkFmgge30zMrr4dukd//JBS3uzwwyqUizsVjXeT/DPs3onaccILGACdUu9Oro/s0BKY4cf94fc1u0Qfloz/64MMO+oALECJn3sxO/DQNbpb9a5n1a+cWoubP2dyxauNPlh6ZJbVZdHxNuuI0QDZx19iAiy46MNLoTrXzl1BsRsGApJiH09mJd2hZ8fATqx4aJFMFq7KTnaUzfrijw02yVJ3vRTdI00e8xXj/CRgoQJCaJKed8tEH1v80pEMgmKHirM9dCMtvVbs3q/71IxjBYsWyCh0unLdsZnT/WsQBSUmkUSYmUk/8y/RAACPUpopmmYJM5tSBhrfS3p9D9TVYOcgUFE91l96FM9+9GvVIKuEEIzonZ46eTYP7Xt4DczZFpQBRfFxbTZ0ZyOpVqk53aC64mLarvQ+w/6lhv06Elbqcf6u7vFrurevgs4CRKCDB57O51cBO0VsH90hvC0kRsmmjPIkIGJDqmoLjEsALH94pdz8KaVsU3IQsdV7rnLzovp/vXA6+A7oEqOud17tHzxYHG15sGhNFNgWxQG+NCtNX/T2ucAE4KYoUqArVg3L7Yx1+YRoCRlilBSz9MC5+N9+5rsO1wEqUkQnHwrHzHrrD/m3TTkcwH1vRJNim2afkpzUvll9+5Fh+pkZPxL868jmQwEMfXa/2PoqpR4BuTkud07MrP/fhTrH914wDqw83Cuq+EY+eyXvrKL/K6FGESLrqTRJENeFP46zaxJyxzKc0L5b1nHleIE/sf7seCvbYHlS/kVJpo9j7g+efE6PaiyvNZys/siNnivtXYv73ABdIMWkpWz6PlNS70eF+fQ5uogmB1uY1WQVM1RUFPD7nUcChH6zlvcuGLbdEkQzqnuscv5hGG/5oLWrPmcxYpYjZc3HhZL59E+VuICFKDkIGZ5tI2QTDsZmNoecL/1F++ZF4vkZABNEGAgFDFPfynUvM12PdvgCrcCxb+bHF5dH990P1VbBKhCt4ON5ZOq/iwPu3MuWUkV4fc9R6KdHa/FQl/jGICLgcrHcNh+XgarX3cafqRYBA8lDNrXaOX6j6G977IuPQBANKZDbz/Wx+pdhes/TQWBEimxRKmQGiEzC1TjeVr8AAb8NCqfJe8eD3lv8joqQTHtxOdI7/hNYtd69ZuUlUIiiKx7PlC6nojQbr5CFQElVtX2q5Wt33qHPKKyX5ApjLKVKibadHl1PvakSfFKASXR1ZjYurxf49HKx1sG/m7kg+m82ft/nTw96/4Fuik3KTE26QAeagVDtC6xHTiUgYBHCo/MZo67dB94yVHGL0eLr72s/cUD68Fqt7ATmARCJ7I1u6UA4fFP3bMzjkE3G+PkHVrNGW0/UPnxWQXiWawywCJXAw2rmC4WcMBVKQQslFzL3NudVq9xb3/2aeyCBnwlyYf5vZybT1m47v0QJJ96b7qbZ1MKarup3QSv+bgu6/LRif5PxYIyhGiICXh1vbm7fnqiieiB6krndOL5z4RZmO9h5sWWHJThmrSjF1vr208t5wwP3eIGKBflSAgXBAARS87pKTSAKFACSQBKGmvm7aK63Mtkh65tOn5frzMiPHco0JjcvCYqzVyfO48B3gV4YhEITMut+yhYupyLn4JuePOMpEORhnvse5d8rhXVt5DzwAozyCNtHeVwDq/i+AAAD0F+TtV5P/JQ9z9CQqgZvQFlgCORCAAC4Cy8AQ2oXlcEd9rtEyMI80QNgDSsCA7N8nb/jXJwEDiHE8mCIIyL4BdVCKv6v/R1MAAAAASUVORK5CYII='
│   },
│   page_content='這張流程圖顯示了一個簡單的製程流程，包括以下步驟：\n\n1.  **開始**：流程從左側的「開始」標誌開始，表示流程的起點。\n2.  **步驟1**：第一個步驟是「步驟1」，表示流程的第一個任務或動作。\n3.  **步驟2**：第二個步驟是「步驟2」，表示流程的第二個任務或動作。\n4.  **步驟3**：第三個步驟是「步驟3」，表示流程的第三個任務或動作。\n5.  **步驟4**：第四個步驟是「步驟4」，表示流程的第四個任務或動作。\n6.  **步驟5**：第五個步驟是「步驟5」，表示流程的第五個任務或動作。\n7.  **步驟6**：第六個步驟是「步驟6」，表示流程的第六個任務或動作。\n8.  **步驟7**：第七個步驟是「步驟7」，表示流程的第七個任務或動作。\n9.  **步驟8**：第八個步驟是「步驟8」，表示流程的第八個任務或動作。\n10. **步驟9**：第九個步驟是「步驟9」，表示流程的第九個任務或動作。\n11. **步驟10**：第十個步驟是「步驟10」，表示流程的第十個任務或動作。\n12. **步驟11**：第十一個步驟是「步驟11」，表示流程的第十一個任務或動作。\n13. **步驟12**：第十二個步驟是「步驟12」，表示流程的第十二個任務或動作。\n14. **步驟13**：第十三個步驟是「步驟13」，表示流程的第十三個任務或動作。\n15. **步驟14**：第十四個步驟是「步驟14」，表示流程的第十四個任務或動作。\n16. **步驟15**：第十五個步驟是「步驟15」，表示流程的第十五個任務或動作。\n17. **步驟16**：第十六個步驟是「步驟16」，表示流程的第十六個任務或動作。\n18. **步驟17**：第十七個步驟是「步驟17」，表示流程的第十七個任務或動作。\n19. **步驟18**：第十八個步驟是「步驟18」，表示流程的第十八個任務或動作。\n20. **步驟19**：第十九個步驟是「步驟19」，表示流程的第十九個任務或動作。\n21. **步驟20**：第二十個步驟是「步驟20」，表示流程的第二十個任務或動作。\n22. **步驟21**：第二十一個步驟是「步驟21」，表示流程的第二十一個任務或動作。\n23. **步驟22**：第二十二個步驟是「步驟22」，表示流程的第二十二個任務或動作。\n24. **步驟23**：第二十三個步驟是「步驟23」，表示流程的第二十三個任務或動作。\n25. **步驟24**：第二十四個步驟是「步驟24」，表示流程的第二十四個任務或動作。\n26. **步驟25**：第二十五個步驟是「步驟25」，表示流程的第二十五個任務或動作。\n27. **步驟26**：第二十六個步驟是「步驟26」，表示流程的第二十六個任務或動作。\n28. **步驟27**：第二十七個步驟是「步驟27」，表示流程的第二十七個任務或動作。\n29. **步驟28**：第二十八個步驟是「步驟28」，表示流程的第二十八個任務或動作。\n30. **步驟29**：第二十九個步驟是「步驟29」，表示流程的第二十九個任務或動作。\n31. **步驟30**：第三十個步驟是「步驟30」，表示流程的第三十個任務或動作。\n32. **步驟31**：第三十一個步驟是「步驟31」，表示流程的第三十一個任務或動作。\n33. **步驟3

In [ ]:
# vectorstore_image = FAISS.from_documents(documents=documents, embedding=embeddings_model)

![image](https://drive.google.com/uc?id=1VO4bYbhjp1rAfvX3RNprZG4Vm566K3AY)


In [ ]:
# vectorstore.merge_from(vectorstore_image)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
retriever.invoke("登入號碼")

[Document(id='2678d1e7-75eb-4263-a72b-91ca61705621', metadata={'subtitle': '膠泵可直接動作。'}, page_content='- (5) 設定每日自動開電加熱的時間。\n- (6) 設定保溫開關。  \n<!-- image -->[/content/sample_data/output/output-picture-2.png]  \n<!-- image -->[/content/sample_data/output/output-picture-3.png]  \n(b)  \n時間設定 設定各時間參數 :  \n- 2. 操作登入：於開機首頁選取＜操作登入＞\n- (1) 輸入本公司給予您的使用者 ID 與密碼變更使用權限  \n管理者－ ID:1 PW:2222  \n產線使用者－ ID:0 無需密碼 僅能操作不能修改任何參數  \n- (2) 依權限開放設定功能，達到管理的效果。  \n<!-- image -->[/content/sample_data/output/output-picture-4.png]  \n<!-- image -->[/content/sample_data/output/output-picture-5.png]'),
 Document(id='0fa61918-f2f9-4b2f-8b5a-f6c19b4d2e00', metadata={}, page_content='<!-- image -->[/content/sample_data/output/output-picture-1.png]'),
 Document(id='dc8bce0f-7553-4770-a4ef-0457ea78b0c0', metadata={'subtitle': '1. 料管溫度操作及設定 : 根目錄下＜溫度→溫度設定＞'}, page_content='- (1) 各段溫控溫度與狀態  \nH1= 射嘴溫控。  \n- H2= 射嘴延長溫控。\n- H3= 料鍋溫控。\n- (2) 設定各段溫度目標與偏差限制。\n- (3) 設定各段溫度保溫溫度。\n- (4) 設定各段溫度是否使用及檢查 注意。 : OFF 時不加溫也不檢查溫度，'),
 Doc

In [ ]:
from langchain_core.runnables import RunnablePassthrough,RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatGroq(
     model="gemma2-9b-it",
     temperature=0,
     max_tokens=None,
     timeout=None,
     max_retries=2,
 )

parallelChain = RunnableParallel(context=retriever, question=RunnablePassthrough())

template = """
"請根據以下提供內容回答問題，若提供內容無法回答問題，則回答不知道。\n"
    "{context}\n"
    "問題: {question}"
"""

prompt = ChatPromptTemplate.from_template(template)

Chain = parallelChain | prompt | llm | StrOutputParser()

In [ ]:
# !pip install -q grandalf

In [ ]:
res = Chain.invoke("天氣如何")
print(res)

不知道。 



# 註冊並登入 LINE Developer
LINE Developers：https://developers.line.biz/zh-hant/

# 建立 Provider
登入後，點選 Create 建立一個 Provider ( 表示開發 LINE BOT 後所在的群組 )

輸入名稱，點擊 Create 就能建立 Provider。

# 建立 Channel
建立 Provider 後，選擇 Channels 頁籤，點選「Create a new channel」建立頻道，一個頻道表示一個聊天機器人。

選擇「Message API」，建立聊天訊息專屬的頻道，建立時需要輸入頻道的名稱和描述述，以及使用下拉選擇所在的地區與類別。

建立完成後，在 Provider 裡就會看到出現了剛剛建立的 Channel。

# LINE 官方帳號設定
建立 Channel 後，前往「LINE 官方帳號管理頁面」。

LINE 官方帳號管理頁面：https://tw.linebiz.com/login/

# 註冊 ngrok 取得 token
ngrok 網站：https://ngrok.com/

登入後，從左側選單點擊 Your Authtoken，會出現一段串接 ngrok 服務所使用的 token

# 安裝 ngrok

In [ ]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2025-03-15 08:03:23--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 35.71.179.82, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  18.5MB/s    in 0.7s    

2025-03-15 08:03:24 (18.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
ngrok.set_auth_token(userdata.get('ngrok_token'))
ngrok_tunnel = ngrok.connect(addr='5000', proto='http', bind_tls=True)

print(' * Tunnel URL:', ngrok_tunnel.public_url)



Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
 * Tunnel URL: https://f8ef-35-192-65-131.ngrok-free.app


In [ ]:
pip install --ignore-installed blinker

  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)


In [ ]:
!pip install -q Flask

In [ ]:
from flask import Flask
from pyngrok import ngrok

app = Flask(__name__)
port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\" ")


@app.route("/")
def home():
    return "Hello Colab!!"

app.run()

 * ngrok tunnel "https://a785-35-192-65-131.ngrok-free.app" -> "http://127.0.0.1:5000" 
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from flask import Flask, request
from pyngrok import ngrok
import json
import requests
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import PyPDF2

# 建立 Flask app
app = Flask(__name__)
port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\" ")

# LINE 的 access token 和 secret
access_token = 'I3g9CfT+VVx/S07wWMMBA8kwCrGfnuffiVg8exDqN/lNoC/FBJnFw4fdkf9zUWsVpvoyOFFnzjF2jvKvSQM6qEpSTkcFWngHScKHH7dDPIT167rl/b9Pv3CqWzwPp0EUd97qXZHixqYK7mOd3ROTCwdB04t89/1O/w1cDnyilFU='
secret = '4deefa2ae3af88760f33d172916784ea'

# 初始化 LINE BOT API 和 Webhook Handler
line_bot_api = LineBotApi(access_token)
handler = WebhookHandler(secret)


@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)                    # 取得收到的訊息內容
    try:
        json_data = json.loads(body)                         # json 格式化訊息內容
        signature = request.headers['X-Line-Signature']      # 加入回傳的 headers
        handler.handle(body, signature)                      # 綁定訊息回傳的相關資訊

        # 取得 LINE 收到的文字訊息
        msg = json_data['events'][0]['message']['text']
        tk = json_data['events'][0]['replyToken']            # 取得回傳訊息的 Token

        # 使用 Chain 回答問題
        answer = Chain.invoke(msg)

        # 使用 LINE Bot 回傳答案
        line_bot_api.reply_message(tk, TextSendMessage(answer))  # 回傳訊息
        print(msg, tk, answer)                                   # 印出內容
    except InvalidSignatureError:
        print("Invalid signature error")
    except Exception as e:
        print("Error: ", e)
        print(body)                                          # 如果發生錯誤，印出收到的內容
    return 'OK'                                              # 驗證 Webhook 使用，不能省略

if __name__ == "__main__":
    app.run()

 * ngrok tunnel "https://b184-35-192-65-131.ngrok-free.app" -> "http://127.0.0.1:5000" 
 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-40-a47a696ba64d>:23: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
<ipython-input-40-a47a696ba64d>:24: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(secret)
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 08:10:40] "POST / HTTP/1.1" 200 -


Error:  list index out of range
{"destination":"U5056bb9a90eebf34938ac20846516633","events":[]}


<ipython-input-40-a47a696ba64d>:43: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(tk, TextSendMessage(answer))  # 回傳訊息
INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 08:10:48] "POST / HTTP/1.1" 200 -


哈囉 93dee1bd450a40fc984bd0ab11c8a61b 哈囉！ 



...

登入者密碼 9f10e9f05219448eaaab8bb34668c7d2 管理者－ ID:1 PW:2222 



INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 08:11:19] "POST / HTTP/1.1" 200 -


Error:  'message'
{"destination":"U5056bb9a90eebf34938ac20846516633","events":[{"type":"follow","follow":{"isUnblocked":false},"webhookEventId":"01JPCDQ21HP0BVBVWP15FV75PQ","deliveryContext":{"isRedelivery":false},"timestamp":1742026278570,"source":{"type":"user","userId":"Ubb1a486ce54f76ec138493aa015b67ba"},"replyToken":"de043601561f426f89531e51eb188c3f","mode":"active"}]}


<ipython-input-40-a47a696ba64d>:43: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(tk, TextSendMessage(answer))  # 回傳訊息
INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 08:12:49] "POST / HTTP/1.1" 200 -


如何設定射出各段壓力 ea052dcb735341a6a7688bf762b5cdb0 根據提供的內容，射出各段壓力設定方法為：

"(1) 設定射出各段壓力 kg/cm2 。"  



![image](https://drive.google.com/uc?id=127SR1BYGBlsWkbtf2XJltZtDv9GkrDo6)